# Functions

In [8]:
from prettyprint.prettyprint import pp
from nltk import wordpunct_tokenize
from hunspell import Hunspell
import re
import numpy as np
import string
from langdetect import detect, detect_langs
from langdetect import DetectorFactory
DetectorFactory.seed = 0

from cytoolz.itertoolz import accumulate
import copy

In [9]:
for l in ['hunspell','nltk','numpy','cytoolz','prettyprint','langdetect']:
    streexe = "import %s;print %s.__name__ +  '=' + %s.__version__"%(l,l,l)
    try:
        exec streexe
    except:
        print 'No version for :: ',l


No version for ::  hunspell
nltk=3.2.4
numpy=1.13.3
cytoolz=0.8.2
prettyprint=0.1.5
No version for ::  langdetect


In [10]:
to_be_replaced = re.compile('[-",.;!()\d\+\[\]\s\%\/]')
clean_bl = re.compile(' {1,10}')

hunspell_dict_dir = 'Hunspell_dict/dictionaries'
lang_enchant = {'de':Hunspell('de_DE',hunspell_dict_dir),
                'fr':Hunspell('fr_FR',hunspell_dict_dir),
                'en':Hunspell('en_GB',hunspell_dict_dir),
               'it':Hunspell('it_IT',hunspell_dict_dir),
               'pt':Hunspell('it_IT',hunspell_dict_dir),
               'us':Hunspell('en_US',hunspell_dict_dir)}

def find_languages(input_str, word_dim=3):
    
    tokens = wordpunct_tokenize(input_str)
    tokens_indexed = zip(tokens,range(len(tokens)))
    words_cleaned = map(lambda x: to_be_replaced.sub(' ',x),tokens)
    words_indexed = zip(words_cleaned,range(len(words_cleaned)))

    lang_dict = {}
    for w_dim in range(word_dim,15):
        par = filter(lambda x: len(x[0]) >= w_dim,words_indexed)
        if par:
            par = string.join(zip(*par)[0],' ')
        else:
            continue
        r1 = detect_langs(par)
        
        for r2 in r1: #= reduce(lambda y,z: z if z.prob > y.prob else y,r1)
            if r2.lang not in lang_dict: 
                lang_dict[r2.lang] = (r2.prob,w_dim)
            else:
                lang_dict[r2.lang] = (r2.prob,w_dim) if r2.prob > lang_dict[r2.lang][0] else lang_dict[r2.lang]
    
    print "DETECTED LANGUAGES :: ", lang_dict
    
    return lang_dict



dict_list = ['en','fr','de','it']

def find_lang_change(input_str, parag_dim=5, word_dim=3, start=0):
    
    #dict_list = ['en','fr','de','it']
    
    tokens = wordpunct_tokenize(input_str)
    tokens_indexed = zip(tokens,range(len(tokens)))
    words_cleaned = map(lambda x: (clean_bl.sub(' ',to_be_replaced.sub(' ',x[0])),x[1]),tokens_indexed)
    
    words = filter(lambda x: len(x[0].strip()) > word_dim,words_cleaned)  
     
    #vv_01 = np.array( [map(lambda x: int(lang_enchant[l].spell(x[0].lower())),words) for l in dict_list]).T
    
    vv_01 = np.array( [map(lambda x: int(lang_enchant[l].spell(x[0].lower())),words) for l in dict_list]).T
    
    old_err_state = np.seterr(divide='raise')
    ignored_states = np.seterr(**old_err_state)
    vv_01_rsum = np.divide(1.0,vv_01.sum(axis=1)[:,np.newaxis])
    vv_01_rsum = np.where(np.isinf(vv_01_rsum),0,vv_01_rsum)
    
    prob = (vv_01*vv_01_rsum).sum(axis=0)/vv_01_rsum.sum()
    lang_prob =  dict(zip(dict_list,prob))
    
    #print '\n', lang_prob, '\n'
    
    vv_01_w = vv_01*prob
    
    #print bn.move_sum(vv_01_w,window=5,axis=0)
    
    vv_red = np.array([vv_01[x:x+parag_dim].sum(axis=0)*prob for x in range(0,len(vv_01)-parag_dim,parag_dim)])
    #print vv_red
    
    chng_pnt = []
    
    start = 0
    for x in range(vv_red.shape[0]):
        if start == 0:
            prev_argmax = np.argmax(vv_red[x])
            idx_1 =  x*parag_dim 
            start = 1
        elif np.max(vv_red[x]) == 0:
            curr_argmax = prev_argmax
        else:
            curr_argmax = np.argmax(vv_red[x])
            
            if curr_argmax <> prev_argmax and lang_prob[dict_list[curr_argmax]] > 0.10:
                idx = [idx_1,x*parag_dim]
                chng_pnt.append((prev_argmax,idx))
                prev_argmax = curr_argmax
                idx_1 =  x*parag_dim

    chng_pnt.append((prev_argmax,[idx_1,len(vv_01)]))            
    chng_pnt = map(lambda x: (dict_list[x[0]],x[1]),chng_pnt)
    
    reduced_chng_pnt = [] 
    
    for x in range(len(chng_pnt)):
        if x == 0:
            last_cp = chng_pnt[x]
        elif chng_pnt[x][1][1]-chng_pnt[x][1][0] <= parag_dim:
            continue
        elif chng_pnt[x][0] == last_cp[0]:
            last_cp = (last_cp[0],[last_cp[1][0],chng_pnt[x][1][1]])
        else:
            reduced_chng_pnt.append(last_cp)
            last_cp = chng_pnt[x]
    last_cp = (last_cp[0],[last_cp[1][0],len(tokens)])        
    reduced_chng_pnt.append(last_cp)
    
    #print reduced_chng_pnt       
    for x in range(len(reduced_chng_pnt)-1):
        
            
        if reduced_chng_pnt[x][1][1] >= 2*parag_dim:
            ind_1 = reduced_chng_pnt[x][1][1] - 2*parag_dim
        elif reduced_chng_pnt[x][1][1] >= parag_dim:
            ind_1 = reduced_chng_pnt[x][1][1] - parag_dim 
        else:
            ind_1 = reduced_chng_pnt[x][1][0]
        
        words_2 = words[ind_1:reduced_chng_pnt[x][1][1]+1*parag_dim] 
        
         
        ori_w = tokens_indexed[words_2[0][1]:words_2[-1][1]]
        has_full_stop = filter(lambda x: x[0]=='.',ori_w)
        wchecked_1 = 0
        if has_full_stop:
            new_index = has_full_stop[0][-1]+1
            chp_to_check = copy.copy(reduced_chng_pnt[x+1])
            chp_to_check[1][0] = new_index
            #print 'chp_to_check ....',chp_to_check
            
            www =  filter(lambda x: len(x[0].strip()) > word_dim,
                                           words_cleaned[new_index:new_index+parag_dim])
            wchecked_1  = np.sum(map(lambda x: int(lang_enchant[chp_to_check[0]].spell(x[0])),www))
            #print 'words .... ', www
            #print 'wchecked_1...... ',wchecked_1
        if wchecked_1 == 0:
            lan_start = reduced_chng_pnt[x][0]
            lan_end = reduced_chng_pnt[x+1][0]
            fil = lambda x: int(lang_enchant[lan_start].spell(x[0]) or bool(to_be_replaced.match(x[0]))) 
            new_index = np.argmax(list(accumulate(lambda y,x: 0 if (x == 0 or y == 0) else y+1,map(fil,ori_w))))
            new_index = ori_w[new_index][1] + 1
            #print 'new_index ....... ' ,new_index
             
        reduced_chng_pnt[x][1][1] = new_index
        reduced_chng_pnt[x+1][1][0] = new_index
    
        
    text = {}
    for k,v in reduced_chng_pnt:
        text[k] = (string.join(tokens[v[0]:v[1]],' '))
     
    return lang_prob,text

    
    

# Profiling and Test

## test strings

In [11]:
str_1_en = u"""
In mathematics, a simplicial complex is a set composed of points, line segments, triangles, 
and their n-dimensional counterparts (see illustration). Simplicial complexes should not be 
confused with the more abstract notion of a simplicial set appearing in modern simplicial homotopy theory. 
The purely combinatorial counterpart to a simplicial complex is an abstract simplicial complex."""

str_1_it = u"""
Un caro saluto a te che, in Italia, ci stai leggendo. 
Sembra che Wikipedia ti piaccia molto; è fantastico! 
Anche se è un po’ imbarazzante chiedertelo, questo mercoledì avremmo bisogno del tuo aiuto. 
Noi non siamo venditori. Siamo bibliotecari, archivisti e persone del mondo dell’informazione."""

str_1_de = u"""
Die Magendrehung (Torsio ventriculi, Dilatatio et Torsio ventriculi) ist eine Krankheit, bei der 
sich der Magen um die eigene Längsachse dreht. Die Ursache der Erkrankung 
ist nicht bekannt. Eine Magendrehung tritt vor allem bei älteren Hunden großer 
Hunderassen auf, selten sind kleine Hunde, Katzen und Meerschweinchen betroffen. 
Die Magendrehung zeigt sich in Zunahme des Bauchumfangs, 
Unruhe und vergeblichen Versuchen zu erbrechen."""

str_en = """
Thorium is a chemical element with symbol Th 
and atomic number 90. A reactive actinide metal, 
its chemistry is dominated by the +4 oxidation state; it is silvery and 
tarnishes black in air, forming the dioxide. 
It is malleable and moderately hard, with a high melting point. 
Its most stable isotope, 232Th, is weakly radioactive with a half-life of 14.05 billion years, roughly the age of the universe: 
it decays very slowly to stable lead.
"""
str_de = u"""
Thoriumverbindungen finden sich häufig in Monazitsanden (Ce,La,Nd,Th) [PO4] + 4...12% ThO2, 
im mit Zirkon isomorphen Mineral Thorit ThSiO4 sowie in Thorianit (Th,U)O2. Auch Titanit 
und Zirkon selbst enthalten geringere Mengen Thorium.
In der Erdkruste kommt Thorium mit einer Häufigkeit von 7 bis 13 mg pro 
kg vor; damit ist es doppelt bis dreimal so häufig wie Uran. Generell ist das Element aufgrund seines 
lithophilen Charakters in geringen Mengen in fast allen silikatischen Gesteinen vertreten.[16]
Die weltweit jährlich für die Stromerzeugung verwendete Kohle enthält unter anderem etwa 10.000 
t Uran und 25.000 t Thorium, die entweder in die Umwelt 
gelangen oder sich in Kraftwerksasche 
und Filterstäuben anreichern.[17]
"""

str_fr = u"""
Lorsqu’il est pur, le thorium est un métal gris-blanc qui conserve son lustre 
pendant plusieurs mois, grâce à l'oxyde qui le protège. Toutefois, 
quand il est exposé à l'oxygène, le thorium ternit lentement dans l'air, 
devient gris et finalement noir.
L'oxyde de thorium (ThO2) est l'un des meilleurs matériaux réfractaires avec une 
température de fusion de 3 300 °C10.
Le thorium métal en poudre est souvent pyrophorique et doit être manipulé avec soin. 
Chauffé dans l'air, des copeaux de thorium peuvent s'enflammer et brûler 
brillamment avec une lumière blanche. Le thorium est l'élément 
qui a la plus grande plage de température pour son état liquide : 
3033 K entre son point de fusion et son point d'ébullition 
(à pression atmosphérique)."""

sstr_en = """
Thorium is a chemical element with symbol Th 
and atomic number 90. A reactive actinide metal, 
its chemistry is dominated by the +4 oxidation state
"""
sstr_de = u"""Thoriumverbindungen finden sich häufig in Monazitsanden (Ce,La,Nd,Th) [PO4] + 4...12% ThO2, 
im mit Zirkon isomorphen Mineral Thorit ThSiO4 sowie in Thorianit (Th,U)O2. Auch Titanit 
und Zirkon selbst enthalten geringere Mengen Thorium.
"""

sstr_fr = u"""
Lorsqu’il est pur, le thorium est un métal gris-blanc qui conserve son lustre 
pendant plusieurs mois, grâce à l'oxyde qui le protège. Toutefois, 
quand il est exposé à l'oxygène, le thorium ternit lentement dans l'air, 
devient gris et finalement noir."""

big_big_str = (str_fr + ' ' + str_de + ' ' + str_en)
#print big_big_str

big_str = (sstr_fr + ' ' + sstr_de + ' ' + sstr_en)
#print big_str

big_str_1 = ( str_1_en + ' ' + str_1_de + ' ' + str_1_it)
#print big_str_1

big_str_2 = (sstr_en + ' ' + sstr_de + ' ' + sstr_fr)
#print big_str_2

big_str_3 = (sstr_en + ' ' + str_1_it + ' ' + sstr_fr + ' ' + sstr_de)
#print big_str_3

str_p_1 = u"""
TIMING IMPACT ALLIGNEMENT - ACC. VERTE LEGI - DA CAPITALIA -DIP.I COASE ""LIQUIDAZIONI"" ELENCO 31/12/2009 N. 2 ULTERIORI ACC. DI  90.000,00 E 30.000,00 - 23.2.2010  .1.322,78 FATT.4 PERULLI 1.6.2010 -  2.911,66 INTEGRAZIONE ACC. PER GIRO FONDI DA PARTITE TRANS.PERSONALE 1.4.2010 -  197,39 PAGAMENTO AVV.E & A SANGREGORIO -  19.317,00 PAG. F24 INPS 23.4.2010 -  593,00 + 23,72 -(RETTIFI 31/12/2010 IN 592,00 E 23,68) PAG. FATT.N. 6 - 23.3.2010 TITLED_NAME_1030560 14.5.2010 -  2.000,00 PAG. A/C FULL_NAME_1030561 -  20.5.2010 -  1.864,71 PAG. EMOLUMENTI.  23.7.2010 -  705,00  ARR.TI EMOLUMENTI DI MAGGIO 23.7.2010 -  755,00 TRATT.FINE RAPPORTO MAGGIO - 23.7.2010 -  20,93 MAGG.PAG. PER GODAKANDE  23.7.2010 -  518,27 MAGG.PAG. MALLAWA  -  4.10.2010 -  4.174,48 PAG. FULL_NAME_1030562 - MEDAS M.ROSARIA 11.11.2010 -  5.170,50 PAG. FULL_NAME_1030563 - 2.11.2010 -  2.745,13 CONTRIBUTI PREVIDENZIALI 30.11.2010 -  5.681,24 TFR - 2.12.2010 -  18,00 DIFF. - 1.3.2011 -   771,30-70,70 MENSILI PREGRES./COMP.FINE RAPPORTO - 03.01.2011 -  110,74 GIRO DA FONDO TFR   - 1.4.2011  236,80 / 9,47 FATT. 26 29.3.2011 FULL_NAME_1030564 - 2.5.2011  1.071,59 GIRO PER PAGAMENTI - 6.5.2011  3.289,89 STUDIO LEG.ANNAGRAZIA CO  FATT.15 -  1.909,00 SANZIONI -1.6.2011 9.150,37 TRATT.FINE RAPPORTO-  1.6.2011 FULL_NAME_1030565 FATT. 28  554,11 FATT. 21  277,06 FATT.27  346,32 FATT.26  345,32 - .2011  2.466,67 FULL_NAME_1030566 FATT. 125 - 1.06.2011  801,74 FULL_NAME_1030567 FATT. N. 27 -  1.041,85 FULL_NAME_1030568 FATT.54 - 31/10/2011  106,81 arret. emolum. ap  763,40 tratt. fine rapp saldo E. 2481,86 SPESE 28/12/2011-E.30685,82  E 1214,00F24 INPS30/12/2011  (01/07/2011 1.071,59 -REITEGRATO ACC.  2/5/2011 GIRO FONDI CAUSA CON.NE GIUDIZIALE WARNAKULAAURYA)"
"""
#-----------------------------------------------------------------------------------------
str_p_2 = u"""
Foreign Payments Corporate Division. In data 14/01/2013 viene ricevuto un ordine di tipo Cash Management in arrivo (MT101) per quale è stato generato in automatico un ordine di pagamento a favore della Societe Generale, Paris , con rif. 111012034761 e l'importo di EUR 1.448.756,25.  Seguito un errore procedurale la lavorazione automatica del flusso telematico non è stata effettuata utilizzando l'acronimo/FN di riferimento ma bensi come ordinante occasionale, di conseguenza la valuta di accredito indicata nell' ordine (14/01/2013) non è stata rispettata, prendendo invece 16/01/2013. Seguito la richiesta ricevuta dalla filiale UniCredit SpA, abbiamo provveduto a richiedere alla controparte la rettifica di valuta, modifica per quale ci è stato richiesto l'importo di EUR 125,00 rappresentando: spese amministrative EUR 100,00 + penali EUR 25,00 (vedesi evento collegato 30095). Abbiamo ricevuto l'autorizzazione di registrare l'importo a DBPOA dal TITLED_NAME_1000517 , struttura GTB Organization.
"""
#-----------------------------------------------------------------------------------------
str_p_3 = u"""Errata esecuzione ordine di vendita cliente FULL_NAME_1000093 operatore TITLED_NAME_1000094 sportello Cento. Dalla ricopertura del titolo &#232; emersa una perdita operativa da tenere a carico Istituto
"""
#-----------------------------------------------------------------------------------------
str_p_4 = u"""
ID 030470 700 - CONTRATTI, 712 - CASSETTA DI SICUREZZA, TRAFUGAMENTO IN CORSO DI RAPINA DI BENI CUSTODITI  CASH BOX RUBBERY ROMA FULL_NAME_1000001 EX BDS BRANCH 1989/07/01
"""

## Profiling

In [29]:
%timeit find_lang_change(big_str_2,parag_dim = 3, word_dim = 3, start = 0)

/Users/flavioc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in divide


100 loops, best of 3: 8.1 ms per loop


In [8]:
%load_ext line_profiler

In [10]:
%lprun -f find_lang_change find_lang_change(big_str_2,parag_dim = 3, word_dim = 3, start = 0)

/Users/flavioc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in divide


## Test

In [12]:
for mstr in [ str_p_1 , str_p_2 , str_p_3 , str_p_4 ]:
    inpstr =  mstr
    find_languages(inpstr, word_dim=3)

DETECTED LANGUAGES ::  {'en': (0.9999967648972486, 13), 'pt': (0.14285898819344972, 4)}
DETECTED LANGUAGES ::  {'it': (0.9999981357413354, 9)}
DETECTED LANGUAGES ::  {'vi': (0.8571402955772294, 12), 'en': (0.14285970441126536, 12), 'it': (0.9999970672026668, 9)}
DETECTED LANGUAGES ::  {'en': (0.9999961957967337, 10), 'pt': (0.14286021621459807, 5)}


In [7]:
#%%timeit
for mstr in [ str_p_1 , str_p_2 , str_p_3 , str_p_4 ]:
    inpstr =  mstr
    
    lp,txt = find_lang_change(inpstr,parag_dim = 3, word_dim = 3, start = 0)
    if 1:
        print '\n' + '*'*30 + '\n'
        print inpstr
        print lp
        for k,v in txt.iteritems():
            print '\n','*'*20,'\n'
            print 'LANG :: ',k,'\n'
            print v

    


******************************


TIMING IMPACT ALLIGNEMENT - ACC. VERTE LEGI - DA CAPITALIA -DIP.I COASE ""LIQUIDAZIONI"" ELENCO 31/12/2009 N. 2 ULTERIORI ACC. DI  90.000,00 E 30.000,00 - 23.2.2010  .1.322,78 FATT.4 PERULLI 1.6.2010 -  2.911,66 INTEGRAZIONE ACC. PER GIRO FONDI DA PARTITE TRANS.PERSONALE 1.4.2010 -  197,39 PAGAMENTO AVV.E & A SANGREGORIO -  19.317,00 PAG. F24 INPS 23.4.2010 -  593,00 + 23,72 -(RETTIFI 31/12/2010 IN 592,00 E 23,68) PAG. FATT.N. 6 - 23.3.2010 TITLED_NAME_1030560 14.5.2010 -  2.000,00 PAG. A/C FULL_NAME_1030561 -  20.5.2010 -  1.864,71 PAG. EMOLUMENTI.  23.7.2010 -  705,00  ARR.TI EMOLUMENTI DI MAGGIO 23.7.2010 -  755,00 TRATT.FINE RAPPORTO MAGGIO - 23.7.2010 -  20,93 MAGG.PAG. PER GODAKANDE  23.7.2010 -  518,27 MAGG.PAG. MALLAWA  -  4.10.2010 -  4.174,48 PAG. FULL_NAME_1030562 - MEDAS M.ROSARIA 11.11.2010 -  5.170,50 PAG. FULL_NAME_1030563 - 2.11.2010 -  2.745,13 CONTRIBUTI PREVIDENZIALI 30.11.2010 -  5.681,24 TFR - 2.12.2010 -  18,00 DIFF. - 1.3.2011 - 

/Users/flavioc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in divide


In [20]:
#%%timeit
for mstr in [ big_str_1 , big_str_2 , big_str_3 ]:
    inpstr =  mstr
    lp,txt = find_lang_change(inpstr,parag_dim = 3, word_dim = 3, start = 0)
    if 1:
        print '\n' + '*'*30 + '\n'
        print inpstr,'\n'
        print lp
        for k,v in txt.iteritems():
            print '\n','*'*20,'\n'
            print 'LANG :: ',k,'\n'
            print v


******************************


In mathematics, a simplicial complex is a set composed of points, line segments, triangles, 
and their n-dimensional counterparts (see illustration). Simplicial complexes should not be 
confused with the more abstract notion of a simplicial set appearing in modern simplicial homotopy theory. 
The purely combinatorial counterpart to a simplicial complex is an abstract simplicial complex. 
Die Magendrehung (Torsio ventriculi, Dilatatio et Torsio ventriculi) ist eine Krankheit, bei der 
sich der Magen um die eigene Längsachse dreht. Die Ursache der Erkrankung 
ist nicht bekannt. Eine Magendrehung tritt vor allem bei älteren Hunden großer 
Hunderassen auf, selten sind kleine Hunde, Katzen und Meerschweinchen betroffen. 
Die Magendrehung zeigt sich in Zunahme des Bauchumfangs, 
Unruhe und vergeblichen Versuchen zu erbrechen. 
Un caro saluto a te che, in Italia, ci stai leggendo. 
Sembra che Wikipedia ti piaccia molto; è fantastico! 
Anche se è un po’ imbara

/Users/flavioc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in divide
